In [1]:
import numpy as np
import pandas as pd
import ast


### GPU Acceleration --IGNORE IF NOT USING COLAB--

In [2]:
# Below line allows for GPU acceleration of sci-kit modeling apis
%load_ext cuml.accel

cuML: Accelerator installed.


In [3]:
#checking cuml and IPython versions, some versions are not compatible. cuml 25.06.00 and IPython 7.34.0 are compatible

import IPython
print(f"IPython version: {IPython.__version__}")

try:
    import cuml
    print(f"cuml version: {cuml.__version__}")
except ImportError:
    print("cuml is not installed or could not be imported.")

IPython version: 7.34.0
cuml version: 25.06.00


### If you are mounting the files from drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Using CSV

In [5]:
main_combined_df_path = '/content/drive/MyDrive/final combined data/all_features_df.csv'
# main_combined_df_path = 'all_features_df.csv'
df = pd.read_csv(main_combined_df_path)
print("Finished reading CSV, now converting claim_text column to python list objects")
df['claim_text'] = df['claim_text'].apply(ast.literal_eval)



Finished reading CSV, now converting claim_text column to python list objects


In [6]:
pd.set_option('display.max_columns', None)
print(df.columns)
print(len(df))
df.head()

Index(['application_number', 'y_approved', 'y_101', 'y_102', 'y_103', 'y_112',
       'claim_text', 'entity_size_MICRO', 'entity_size_SMALL',
       'entity_size_UNDISCOUNTED',
       ...
       'art_unit_cat_Biotech and Organic Chemistry',
       'art_unit_cat_Chemical and Materials Engineering',
       'art_unit_cat_Communications',
       'art_unit_cat_Computer Architecture and Software',
       'art_unit_cat_Design',
       'art_unit_cat_Mechanical Engineering, Manufacturing, and Products',
       'art_unit_cat_Networking, Multiplexing, Cable, and Cybersecurity',
       'art_unit_cat_Other',
       'art_unit_cat_Semiconductors/Memory, Circuits/Measuring and Testing, Optics/Photocopying, Printing/Measuring and Testing',
       'art_unit_cat_Transportation, Construction, Electronic Commerce, Agriculture, National Security and License and Review'],
      dtype='object', length=226)
3909923


,application_number,y_approved,y_101,y_102,y_103,y_112,claim_text,entity_size_MICRO,entity_size_SMALL,entity_size_UNDISCOUNTED,aia_first_to_file_False,aia_first_to_file_True,groupartunitnumber_x,number_of_oa,unique_rej_types,num_office_actions,oa_span_days,mean_gap_days,min_gap_days,max_gap_days,years_since_pub,country_AD,country_AE,country_AG,country_AI,country_AL,country_AM,country_AN,country_AO,country_AR,country_AS,country_AT,country_AU,country_AW,country_AZ,country_BA,country_BB,country_BD,country_BE,country_BF,country_BG,country_BH,country_BI,country_BM,country_BN,country_BO,country_BR,country_BS,country_BV,country_BY,country_BZ,country_CA,country_CB,country_CH,country_CI,country_CK,country_CL,country_CM,country_CN,country_CO,country_CR,country_CU,country_CV,country_CW,country_CY,country_CZ,country_DE,country_DK,country_DM,country_DO,country_DZ,country_EC,country_EE,country_EG,country_EI,country_ER,country_ES,country_ET,country_FI,country_FO,country_FR,country_GA,country_GB,country_GD,country_GE,country_GG,country_GH,country_GI,country_GN,country_GR,country_GT,country_HB,country_HK,country_HM,country_HN,country_HR,country_HT,country_HU,country_ID,country_IE,country_IL,country_IM,country_IN,country_IO,country_IQ,country_IR,country_IS,country_IT,country_J,country_JE,country_JM,country_JO,country_JP,country_KE,country_KH,country_KI,country_KN,country_KP,country_KR,country_KW,country_KY,country_KZ,country_LB,country_LC,country_LI,country_LK,country_LT,country_LU,country_LV,country_LX,country_MA,country_MC,country_MD,country_ME,country_MG,country_MH,country_MI,country_MK,country_MM,country_MN,country_MO,country_MR,country_MT,country_MU,country_MX,country_MY,country_MZ,country_N,country_NA,country_NC,country_NE,country_NG,country_NJ,country_NL,country_NO,country_NP,country_NY,country_NZ,country_OM,country_PA,country_PE,country_PH,country_PK,country_PL,country_PR,country_PS,country_PT,country_PY,country_QA,country_RO,country_RS,country_RU,country_SA,country_SC,country_SD,country_SE,country_SG,country_SH,country_SI,country_SK,country_SL,country_SM,country_SN,country_SR,country_SS,country_SV,country_SW,country_SZ,country_T,country_TC,country_TH,country_TI,country_TK,country_TN,country_TO,country_TR,country_TS,country_TT,country_TW,country_UA,country_UK,country_US,country_UY,country_UZ,country_VA,country_VC,country_VE,country_VG,country_VI,country_VN,country_VU,country_WS,country_YR,country_ZA,country_ZW,country_na,art_unit_cat_Biotech and Organic Chemistry,art_unit_cat_Chemical and Materials Engineering,art_unit_cat_Communications,art_unit_cat_Computer Architecture and Software,art_unit_cat_Design,"art_unit_cat_Mechanical Engineering, Manufacturing, and Products","art_unit_cat_Networking, Multiplexing, Cable, and Cybersecurity",art_unit_cat_Other,"art_unit_cat_Semiconductors/Memory, Circuits/Measuring and Testing, Optics/Photocopying, Printing/Measuring and Testing","art_unit_cat_Transportation, Construction, Electronic Commerce, Agriculture, National Security and License and Review"
0,13816709,1,1,1,1,1,"[nonamplifi, detect, polynucleotid, probe, pro...",False,False,True,True,False,1634.0,10,4,10.0,2513.0,279.222222,187.0,464.0,8.933607,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
1,15998927,1,0,1,1,1,"[compound, follow, formula, pharmaceut, accept...",False,True,False,True,False,1624.0,4,3,4.0,568.0,189.333333,106.0,238.0,2.491444,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Manual Vectorizer Creation --IGNORE--

Vectorization does work if you do it from scratch, however it does not save to the joblib file properly for some reason. This is not the preferred way to make a sparse matrix since Karl has provided a vectorizationn object that does work with claims text

In [7]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_features = joblib.load('./tfidf_features.joblib')
# tlidf_vectorizer = joblib.load('./tfidf_vectorizer.joblib')

# suppose texts is a NumPy array (or list) of token lists
# e.g. [['novel', 'catalyst'], ['apparatus', 'for', '...'], ...]

def identity(tokens):
    return tokens

tlidf_vectorizer = TfidfVectorizer(
    analyzer=identity,     # accept pre-tokenized lists
    lowercase=False,       # tokens already normalized
    dtype=float,
    norm="l2"
)

In [8]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

def build_vocabulary(input_df):
    #This builds a vocabulary based on the tokens in the claim_text column
    #it saves the vocabulary to whatever you put in output_vocab_file which should be a .json file

    vocab = set()

    reader = input_df['claim_text']
    print('Creating vocabulary...')
    for token_list in reader:
        vocab.update(token_list)
    print(f"Vocabulary created with {len(vocab):,} unique tokens")

    # Convert set to sorted list for stable saving
    vocab_list = sorted(vocab)

    return vocab_list

def build_claim_vectorizer(vocab):
    """Return a TF-IDF vectorizer with the same settings as fit_tfidf_vectorizer."""
    vectorizer = TfidfVectorizer(
        vocabulary={tok: i for i, tok in enumerate(vocab)},
        analyzer=lambda tokens: tokens,   # accept pre-tokenized lists
        lowercase=False,
        norm="l2",
        dtype=float,
        use_idf=True,

    )
    return vectorizer

# usage
# df['claim_text'] is a column of lists, e.g. ['apparatus', 'comprising', ...]

vocab_list = build_vocabulary(df)
vectorizer = build_claim_vectorizer(vocab_list)
sparse_matrix = vectorizer.fit_transform(df['claim_text'])

Creating vocabulary...
Vocabulary created with 637,906 unique tokens


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'float'> 'dtype' will be converted to np.float64.
  warnings.warn(


In [ ]:
from scipy import sparse
num_docs = len(df['claim_text'])
token_to_idx = vectorizer.vocabulary_
df_counts = np.zeros(len(token_to_idx), dtype=np.int64)

for tokens in df['claim_text']:
    for tok in set(tokens):
        idx = token_to_idx.get(tok)
        if idx is not None:
            df_counts[idx] += 1


idf = np.log((1 + num_docs) / (1 + df_counts)) + 1.0
vectorizer.idf_ = idf
vectorizer._tfidf._idf_diag = sparse.spdiags(idf, diags=0, m=len(idf), n=len(idf))
joblib.dump(vectorizer, './tfidf_vectorizer_v2.joblib')


### Vectorizer Loading -- DONT IGNORE --

In [10]:
import joblib
# sparse_matrix = tlidf_vectorizer.transform(df['claim_text'])
print("loading tfidf_vectorizer...")
tlidf_vectorizer = joblib.load('/content/drive/MyDrive/final combined data/tfidf_vectorizer_for_bryant.joblib')
print("finished loading vectorizer, now creating sparse matrix")
sparse_matrix = tlidf_vectorizer.fit_transform(df['claim_text'])




loading tfidf_vectorizer...
finished loading vectorizer, now creating sparse matrix


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'float'> 'dtype' will be converted to np.float64.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TypeError: descriptor 'split' for 'str' objects doesn't apply to a 'list' object

### SVD of tfidf matrix, Creation of X data and Y data for train-test-split

In [9]:
from sklearn.decomposition import TruncatedSVD
num_dimensions = 100
svd = TruncatedSVD(
    n_components=num_dimensions,
    n_iter=7,
    random_state=70
)
sparse_matrix.shape


(3909923, 637906)

In [10]:
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

extra_np = (
    df.drop(columns=['claim_text', 'application_number', 'y_approved', 'y_101', 'y_102', 'y_103', 'y_112'])
    .apply(pd.to_numeric, errors="coerce")  # convert strings/bools -> numbers
    .fillna(0)                              # replace NaNs
    .to_numpy(dtype=np.float64)             # ensure a numeric dtype
)

extra_sparse = csr_matrix(extra_np)
print(f"Performing TruncatedSVD with {num_dimensions} components...")
tfidf_svd = svd.fit_transform(sparse_matrix)

X_aug_combined = hstack([tfidf_svd, extra_sparse])
X_aug_no_tfidf = extra_sparse


Performing TruncatedSVD with 100 components...


In [11]:
y_rejected = df['y_approved'] == 0
y_rejected = y_rejected.astype(int)
y_has_rej101 = df['y_101']
y_has_rej102 = df['y_102']
y_has_rej103 = df['y_103']
y_has_rej112 = df['y_112']

In [12]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


### Train test split of all data

Below is train test split of data without tfidf sparse matrix

In [13]:
#  getting train and test data for data with no tfidf
from sklearn.model_selection import train_test_split
X_train_rejected_no_tfidf, X_test_rejected_no_tfidf, y_train_rejected_no_tfidf, y_test_rejected_no_tfidf = train_test_split(X_aug_no_tfidf, y_rejected, test_size=0.2, random_state=42)
X_train_has_rej101_no_tfidf, X_test_has_rej101_no_tfidf, y_train_has_rej101_no_tfidf, y_test_has_rej101_no_tfidf = train_test_split(X_aug_no_tfidf, y_has_rej101, test_size=0.2, random_state=42)
X_train_has_rej102_no_tfidf, X_test_has_rej102_no_tfidf, y_train_has_rej102_no_tfidf, y_test_has_rej102_no_tfidf = train_test_split(X_aug_no_tfidf, y_has_rej102, test_size=0.2, random_state=42)
X_train_has_rej103_no_tfidf, X_test_has_rej103_no_tfidf, y_train_has_rej103_no_tfidf, y_test_has_rej103_no_tfidf = train_test_split(X_aug_no_tfidf, y_has_rej103, test_size=0.2, random_state=42)
X_train_has_rej112_no_tfidf, X_test_has_rej112_no_tfidf, y_train_has_rej112_no_tfidf, y_test_has_rej112_no_tfidf = train_test_split(X_aug_no_tfidf, y_has_rej112, test_size=0.2, random_state=42)

In [14]:
#  getting train and test data for data with tfidf
from sklearn.model_selection import train_test_split

X_train_rejected, X_test_rejected, y_train_rejected, y_test_rejected = train_test_split(X_aug_combined, y_rejected, test_size=0.2, random_state=42)
X_train_has_rej101, X_test_has_rej101, y_train_has_rej101, y_test_has_rej101 = train_test_split(X_aug_combined, y_has_rej101, test_size=0.2, random_state=42)
X_train_has_rej102, X_test_has_rej102, y_train_has_rej102, y_test_has_rej102 = train_test_split(X_aug_combined, y_has_rej102, test_size=0.2, random_state=42)
X_train_has_rej103, X_test_has_rej103, y_train_has_rej103, y_test_has_rej103 = train_test_split(X_aug_combined, y_has_rej103, test_size=0.2, random_state=42)
X_train_has_rej112, X_test_has_rej112, y_train_has_rej112, y_test_has_rej112 = train_test_split(X_aug_combined, y_has_rej112, test_size=0.2, random_state=42)

### Modeling

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [16]:
def get_best_random_gb_params(X_train, y_train, scoring = 'f1', n_iter = 20):
    gb = XGBClassifier(random_state=0)
    gb_space = {
        "n_estimators": np.arange(50, 401),
        "learning_rate": np.logspace(-3, 0, 100),
        "max_depth": np.arange(1, 6),
        "min_samples_split": np.arange(2, 21),
    }
    gb_rand = RandomizedSearchCV(
        gb,
        gb_space,
        n_iter=n_iter,
        scoring=scoring,
        cv=5,
        random_state=0,
        n_jobs=-1,
    )
    gb_rand.fit(X_train, y_train)
    gb_best = gb_rand.best_params_
    return gb_best

def get_best_random_rf_params(X_train, y_train, scoring = 'f1', n_iter = 20):
    rf = RandomForestClassifier(random_state=0)
    rf_space = {
        "n_estimators": np.arange(100, 801),
        "max_depth": [None, *np.arange(5, 31)],
        "min_samples_split": np.arange(2, 21),
        "min_samples_leaf": np.arange(1, 11),
        "max_features": ["sqrt", "log2", 0.5, None],
    }
    rf_rand = RandomizedSearchCV(
        rf,
        rf_space,
        n_iter=n_iter,
        scoring=scoring,
        cv=5,
        random_state=0,
        n_jobs=-1,
    )
    rf_rand.fit(X_train, y_train)
    rf_best = rf_rand.best_params_
    return rf_best

def get_best_grid_search_gb_model(X_train, y_train, gb_best, scoring = 'f1'):
    gb_grid = {
        "n_estimators": [gb_best["n_estimators"] - 50,
                        gb_best["n_estimators"],
                        gb_best["n_estimators"] + 50],
        "learning_rate": [gb_best["learning_rate"] * f for f in [0.5, 1.0, 1.5]],
        "max_depth": [gb_best["max_depth"]],
        "min_samples_split": [gb_best["min_samples_split"] - 1,
                            gb_best["min_samples_split"],
                            gb_best["min_samples_split"] + 1],
    }
    gb_grid = {k: [v] if not isinstance(v, list) else v for k, v in gb_grid.items()}
    gb_grid = {k: [val for val in vals if val is not None and val > 0] for k, vals in gb_grid.items()}
    gb_gridsearch = GridSearchCV(
        GradientBoostingClassifier(random_state=0),
        gb_grid,
        scoring=scoring,
        cv=5,
        n_jobs=-1,
    )
    return gb_gridsearch

def get_best_grid_search_rf_model(X_train, y_train, rf_best, scoring = 'f1'):
    rf_grid = {
        "n_estimators": [rf_best["n_estimators"] - 100,
                        rf_best["n_estimators"],
                        rf_best["n_estimators"] + 100],
        "max_depth": [None if rf_best["max_depth"] is None else d for d in [
            rf_best["max_depth"] - 5,
            rf_best["max_depth"],
            rf_best["max_depth"] + 5,
        ] if d is None or d > 0],
        "min_samples_split": [max(2, rf_best["min_samples_split"] - 1),
                            rf_best["min_samples_split"],
                            rf_best["min_samples_split"] + 1],
        "min_samples_leaf": [max(1, rf_best["min_samples_leaf"] - 1),
                            rf_best["min_samples_leaf"],
                            rf_best["min_samples_leaf"] + 1],
        "max_features": [rf_best["max_features"]],
    }
    rf_grid = {k: [v] if not isinstance(v, list) else v for k, v in rf_grid.items()}
    rf_grid = {k: [val for val in vals if val is not None] for k, vals in rf_grid.items()}
    rf_gridsearch = GridSearchCV(
        RandomForestClassifier(random_state=0),
        rf_grid,
        scoring=scoring,
        cv=5,
        n_jobs=-1,
    )
    return rf_gridsearch

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import time

def create_models(X_train, y_train, X_train_gpu = None,rf_params = None, gb_params = None):
    models = {}
    models['LogR'] = LogisticRegression(solver='saga')
    models['LinR'] = LinearRegression()
    if rf_params:
        models['RF'] = RandomForestClassifier(**rf_params)
    else:
        models['RF'] = RandomForestClassifier(n_estimators=100)
    if gb_params:
        models['GB'] = XGBClassifier(**gb_params)
    else:
        models['GB'] = XGBClassifier(n_estimators=100)
    print("Fitting Logistic Regression")
    start_time = time.time()
    models['LogR'].fit(X_train, y_train)
    end_time = time.time()
    print(f"Logistic Regression training time: {end_time - start_time} seconds")
    print("Fitting Linear Regression")
    start_time = time.time()
    models['LinR'].fit(X_train, y_train)
    end_time = time.time()
    print(f"Linear Regression training time: {end_time - start_time} seconds")

    if X_train_gpu is not None:
      print("Fitting Random Forest with Dense GPU Training Data")
      start_time = time.time()
      models['RF'].fit(X_train_gpu, y_train)
      end_time = time.time()
      print(f"Random Forest training time: {end_time - start_time} seconds")

    else:
      print("Fitting Random Forest")
      start_time = time.time()
      models['RF'].fit(X_train, y_train)
      end_time = time.time()
      print(f"Random Forest training time: {end_time - start_time} seconds")

    print("Fitting Gradient Boosting with Dense GPU Training Data")
    start_time = time.time()
    models['GB'].fit(X_train, y_train)
    end_time = time.time()
    print(f"Gradient Boosting training time: {end_time - start_time} seconds")
    return models

In [18]:
# Fill in below with the right data, currently filled in for overall rejections using X values with no tfidf data
X_train = X_train_rejected_no_tfidf
y_train = y_train_rejected_no_tfidf
X_test = X_test_rejected_no_tfidf
y_test = y_test_rejected_no_tfidf

In [19]:
import cupy as cp
print("Converting training data to dense array for GPU...")
start_time = time.time()
X_train_dense_gpu = cp.asarray(X_train.todense())
end_time = time.time()
print(f"Conversion time: {end_time - start_time} seconds")
print("Converting testing data to dense array for GPU")
start_time = time.time()
X_test_dense_gpu = cp.asarray(X_train.todense())
end_time = time.time()
print(f"Conversion time: {end_time - start_time} seconds")

Converting training data to dense array for GPU...
Conversion time: 9.7493257522583 seconds
Converting testing data to dense array for GPU
Conversion time: 6.132742404937744 seconds


In [ ]:
rf_params_random = get_best_random_rf_params(X_train, y_train)
gb_params_random = get_best_random_gb_params(X_train, y_train)
rf_params_grid = get_best_grid_search_rf_model(X_train, y_train, rf_params_random)
gb_params_grid = get_best_grid_search_gb_model(X_train, y_train, gb_params_random)

In [ ]:
rf_params_random = get_best_random_rf_params(X_train_dense_gpu, y_train)


Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: monotonic_cst during cuML estimator initialization
Unused keyword parameter: ccp_alpha during cuML estimator initialization
Unused keyword parameter: m

In [ ]:
gb_params_random = get_best_random_gb_params(X_train, y_train)


In [ ]:
rf_params_grid = get_best_grid_search_rf_model(X_train_dense_gpu, y_train, rf_params_random)


In [ ]:
gb_params_grid = get_best_grid_search_gb_model(X_train, y_train, gb_params_random)

In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix



#fill in rf params and gb params with the best params from randomized search or grid search
rf_params = None
gb_params = None
# get accuracy, F1, precision, recall, and confusion matrix
models = create_models(X_train, y_train, X_train_gpu=X_train_dense_gpu)
for model_name, model in models.items():
    y_pred = model.predict(X_test)

    # Convert continuous predictions to binary for LinearRegression
    if model_name == 'LinR':
        y_pred = (y_pred > 0.5).astype(int)

    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"F1 Score: {f1_score(y_test, y_pred)}")
    print(f"Precision: {precision_score(y_test, y_pred)}")
    print(f"Recall: {recall_score(y_test, y_pred)}")
    print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")
    print("\n")


Fitting Logistic Regression
L-BFGS: max iterations reached
Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
Logistic Regression training time: 9.091042280197144 seconds
Fitting Linear Regression
Linear Regression training time: 5.090709209442139 seconds
Fitting Random Forest with Dense GPU Training Data
Random Forest training time: 14.147544145584106 seconds
Fitting Gradient Boosting with Dense GPU Training Data
Gradient Boosting training time: 9.152363777160645 seconds
Model: LogR
Accuracy: 0.7619059189114881
F1 Score: 0.2855871135088675
Precision: 0.5640535952467564
Recall: 0.19119596378918813
Confusion Matrix: [[558585  28762]
 [157424  37214]]


Model: LinR


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [30]:
pip install xgboost

In [34]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
gb_model = XGBClassifier(n_estimators=100)
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")
print("\n")



Accuracy: 0.796836256449932
F1 Score: 0.46561878781966975
Precision: 0.674199549975161
Recall: 0.35560373616662727
Confusion Matrix: [[553900  33447]
 [125424  69214]]


